In [16]:
%pip install google-cloud-bigquery
%pip install "google-cloud-bigquery[pandas]"
%pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/tanya/Desktop/Data Science Journey/SQL/spheric-passkey-475414-m3-613464412c9f.json"

In [18]:
#we need to create a client object to retrieve information from BigQuery datasets
client = bigquery.Client(project="spheric-passkey-475414-m3")  # ← your project ID

In [19]:
# datasets exist in projects in bigquery. we would like to work with hacker_news dataset contained in bigquery-public-data project
# here we connect to the dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")


# now let's fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [20]:
# this dataset is like a set of tables. so now let's list those tables 
tables = list(client.list_tables(dataset))

# now we want the names of those tables. each of them has an id and this code shows the names of the tables
for table in tables:
    print(table.table_id)

# there is one table!

full


In [21]:
# Let's open a table now by calling it
# first we construct a reference to the table
table_ref = dataset_ref.table("full")

# then through that reference we fetch the table
table = client.get_table(table_ref)

In [22]:
# let's investigate the table. 
# print information on all the columns

table.schema

#each schemafield holds into about a specific column. 
# first is column name
# second is data type 
# third os mode of the column (nullable means allows null values)
# fourth is description of the data of the column 

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

In [23]:
# let's have a look at the first five rows of the full table

client.list_rows(table, max_results = 5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,Adoum_Tech,2,1713995025,2024-04-24 21:43:45+00:00,story,40150086,<NA>,<NA>,<NA>,<NA>
1,None,None,None,True,belter,2,1713995286,2024-04-24 21:48:06+00:00,story,40150135,<NA>,<NA>,<NA>,<NA>
2,None,None,None,True,Rinzler89,1,1713995678,2024-04-24 21:54:38+00:00,story,40150207,<NA>,<NA>,<NA>,<NA>
3,None,None,None,True,stockstobuynow,1,1713995704,2024-04-24 21:55:04+00:00,story,40150212,<NA>,<NA>,<NA>,<NA>
4,None,None,None,True,FLMAN407,1,1713995772,2024-04-24 21:56:12+00:00,story,40150229,<NA>,<NA>,<NA>,<NA>


In [24]:
# the method above also lets us to have information of a specific column
client.list_rows(table, selected_fields = table.schema[:1], max_results = 5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


Let's work with OpenAQ dataset 

In [25]:
dataset_ref_aq = client.dataset("openaq", project = "bigquery-public-data") #build a reference to the dataset

dataset_aq = client.get_dataset(dataset_ref_aq)  #fetch the dataset

tables_aq = list(client.list_tables(dataset_aq))  #list all the tables in openaq dataset

tabs = []
for table in tables_aq: #print the names of the tables
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")
    

There are 1 tables, and their names are: ['global_air_quality']


Now let's return the table columns and rows 

In [26]:
# first let's create a table reference
table_ref_aq = dataset_ref_aq.table("global_air_quality")

# fetch the table through reference of the table
table_aq_gaq = client.get_table(table_ref_aq)

#now let's list the rows
client.list_rows(table_aq_gaq, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


let's fetch the cities in the united states

In [35]:
#let's write our query
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# let's add more columns 
query2 = """
        select city, country
        from `bigquery-public-data.openaq.global_air_quality`
        where country = 'US'
        """

In [37]:
# We need to use the query to return what we want 
query_job = client.query(query2)

# query_job is the subset of the data we filtered and now we turn it into a pandas dataframe called us_cities
us_cities = query_job.to_dataframe()

In [38]:
# let's get the value counts of the dataframe
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [39]:
us_cities

,city,country
0,HOWARD,US
1,HOWARD,US
2,HOWARD,US
3,HOWARD,US
4,HOWARD,US
...,...,...
1421346,Washington-Arlington-Alexandria,US
1421347,Washington-Arlington-Alexandria,US
1421348,Washington-Arlington-Alexandria,US
1421349,Washington-Arlington-Alexandria,US
